## Mengimport library

In [1]:
import torch

Pertama-tama kita akan membuat variabel

In [2]:
# Membuat variabel dengan mengaktifkan ```requires_grad```
x = torch.randn(3, requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
z = z.mean()
print(z)

tensor([-0.3908, -0.2844,  0.4472], requires_grad=True)
tensor([1.6092, 1.7156, 2.4472], grad_fn=<AddBackward0>)
tensor(11.5216, grad_fn=<MeanBackward0>)


## Menghitung gradien

In [3]:
# dz/dx
z.backward()
print(x.grad)

tensor([3.2185, 3.4312, 4.8944])


Gradien diatas dapat dihitung karena nilai dari z berupa skalar dan bukan vektor. Apabila misalnya kita hanya melakukan operasi z = y * y * 3, maka nilai dari z tidak berupa vektor dan tidak dapat dihitung gradiennya. Lalu bagaimana jika ingin melakukan backpropagation terhadap nilai yang berupa vektor?

In [4]:
x = torch.randn(3, requires_grad=True)
y = x * 2
z = y * y * 3

# kita membutuhkan v sebagai array yang akan membantu melakukan backward terhadap z
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([ 1.9805e+00,  1.4602e+01, -1.3995e-04])


## Menghentikan Tracking Gradien

In [5]:
x = torch.randn(3, requires_grad=True)
y = torch.randn(3, requires_grad=True)
z = torch.randn(3, requires_grad=True)
print(x)
print(y)
print(z)

tensor([1.5052, 0.5140, 1.7380], requires_grad=True)
tensor([-0.4040, -1.7743,  0.6048], requires_grad=True)
tensor([ 2.0259,  0.2202, -1.3852], requires_grad=True)


### Cara pertama

In [6]:
x.requires_grad_(False)
print(x)

tensor([1.5052, 0.5140, 1.7380])


### Cara kedua

In [7]:
y = y.detach()
print(y)

tensor([-0.4040, -1.7743,  0.6048])


### Cara ketiga

In [8]:
with torch.no_grad():
    z = x + y
    print(z)

tensor([ 1.1012, -1.2603,  2.3428])


## Gradien yang salah akibat dari backpropagation
Ketika kita melakukan backpropagation dengan epoch yang berjumlah lebih dari satu, maka kita akan mendapatkan nilai gradien yang salah seperti pada kode di bawah ini. Nilai dari gradien terakumulasi pada setiap perulangan epoch.

In [9]:
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


Untuk mengatasi kesalahan di atas, maka kode di bawah ini akan menghilangkan nilai gradien yang salah dengan cara mengosongkan nilai gradien

In [10]:
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print(weights.grad)
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


Hal yang sama juga perlu dilakukan ketika menggunakan optimizer

In [ ]:
weights = torch.ones(4, requires_grad=True)
optimizer = torch.optim.SGD(weights, lr=0.01)
optimizer.step()
# gunakan optimizer untuk mengoptimalkan nilai weights
optimizer.zero_grad()